In [ ]:
import tracemalloc  # For debugging memory usage
from deep_translator import GoogleTranslator
from gtts import gTTS
from playsound import playsound
import speech_recognition as sr
import tempfile
import os

# Enable memory tracking for debugging
tracemalloc.start()

# Translate text using deep-translator (reliable and synchronous)


def translate_text(text, dest_lang, source_lang):
    translator = GoogleTranslator(source=source_lang, target=dest_lang)
    return translator.translate(text)

# Convert translated text to speech and play it


def read_text(text, dest_lang, source_lang):
    otherlang_text = translate_text(text, dest_lang, source_lang)
    print("Translation:", otherlang_text)

    # Create and play a temporary audio file
    with tempfile.NamedTemporaryFile(delete=False, suffix=".mp3") as fp:
        tts = gTTS(text=otherlang_text, lang=dest_lang)
        tts.save(fp.name)
        playsound(fp.name)
        os.unlink(fp.name)


dest_lang = "ta"

# Configure speech recognition
r = sr.Recognizer()
r.pause_threshold = 1  # 3-second silence detection
r.energy_threshold = 200  # Noise sensitivity
i = 1

# Use the microphone as the audio source
with sr.Microphone() as source:
    print("🔊 Adjusting for ambient noise...")
    r.adjust_for_ambient_noise(source, duration=1)  # Dynamic noise calibration

    while True:
        try:
            if i % 2 == 1:
                print("\n🟢 Listening english...")
                audio = r.listen(source)  # Listen with a 5-second timeout
                english_text = r.recognize_google(
                    audio, language="en-IN")  # Recognize speech
                print("You said:", english_text)
                # Translate and speak the recognized text
                read_text(english_text, dest_lang, "en")

            else:
                print("\n🟢 Listening tamil...")
                audio = r.listen(source)  # Listen with a 5-second timeout
                tamil_text = r.recognize_google(
                    audio, language=dest_lang)  # Recognize speech
                print("You said:", tamil_text)
                # Translate and speak the recognized text
                read_text(tamil_text, "en", dest_lang)
            i = i + 1

        except sr.WaitTimeoutError:
            print("🟡 No speech detected (Timeout)")
        except sr.UnknownValueError:
            print("🔴 Could not understand the audio (Unclear Speech)")
            if i % 2 == 1:
                english_text = "Can you repeat it"
                print("You said:", english_text)
                read_text(english_text, dest_lang, "en")
            else:
                translator = GoogleTranslator(source="en", target=dest_lang)
                text = translator.translate("Can you repeat it")
                print("You said:", text)
                read_text(text, "en", dest_lang)
            i = i+1
        except KeyboardInterrupt:
            print("\n🛑 Stopped by user (CTRL+C)")
            break

# Optional: Print memory usage (for debugging purposes)
snapshot = tracemalloc.take_snapshot()
for stat in snapshot.statistics("lineno")[:5]:
    print(stat)

🔊 Adjusting for ambient noise...

🟢 Listening english...
You said: hello I'm Doctor
Translation: வணக்கம் நான் மருத்துவர்

🟢 Listening tamil...
You said: வணக்கம் ஐயா
Translation: Hello sir

🟢 Listening english...
You said: how can I help you
Translation: நான் உங்களுக்கு எவ்வாறு உதவ முடியும்

🟢 Listening tamil...
You said: எனக்கு இரண்டு நாட்களாக ஒரே ஜுரம்
Translation: I have two days of the same jar

🟢 Listening english...
You said: okay I'll give tablets for 2 days please take it
Translation: சரி, நான் 2 நாட்களுக்கு மாத்திரைகள் கொடுப்பேன் தயவுசெய்து அதை எடுத்துக் கொள்ளுங்கள்

🟢 Listening tamil...
You said: உணவுப்பழக்கத்தை ஏதாவது மாற்றிக் கொள்ள வேண்டுமா ஐயா
Translation: Do you want to change the diet sir sir

🟢 Listening english...
You said: no worries you can continue with this one
Translation: இதை நீங்கள் தொடர முடியாது

🟢 Listening tamil...
You said: மிக்க நன்றி ஐயா
Translation: Thank you so much sir

🟢 Listening english...
You said: thank you have a good day
Translation: நன்றி ஒரு நல்